# Importing libraries

In [1]:
from google.colab import drive
import json
import os
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Import Data

In [3]:
import os
import json

# --- percorso assoluto su Kaggle ---
DATA_PATH = "/kaggle/input/fds-pokemon-battles-prediction-2025"

train_file_path = os.path.join(DATA_PATH, "train.jsonl")
test_file_path = os.path.join(DATA_PATH, "test.jsonl")

train_data = []
with open(train_file_path, "r") as f:
    for line in f:
        train_data.append(json.loads(line))

test_data = []
with open(test_file_path, "r") as f:
    for line in f:
        test_data.append(json.loads(line))

print(f"✅ Train data loaded: {len(train_data)} records")


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/fds-pokemon-battles-prediction-2025/train.jsonl'

# Cleaning the Data

In [4]:
train_data.remove(train_data[4877])

In [5]:
# Removing battles where pokemons' level is not == 100

for i, battle in enumerate(train_data):
  squadra1 = battle['p1_team_details']
  for pok in squadra1:
    livello = pok['level']
    if livello < 100:
      print(livello)
      train_data.remove(train_data[i])

print(len(train_data))

55
55
55
55
55
55
55
55
55
55
55
55
85
9986


In [6]:
# LISTA POKEMON
pokemon=[]
for i in train_data:
  for j in i['p1_team_details']:
    if j not in pokemon:
      pokemon.append(j)
print(pokemon)
print(len(pokemon))

#lista solo nomi:
nomi=[]
for i in pokemon:
  nomi.append(i['name'])
print(nomi)
print(len(nomi))

for i in train_data:
  battle=i['battle_timeline']
  for j in battle:
    pok=j['p2_pokemon_state']
    name=pok['name']
    if name not in nomi:
      print(name)

[{'name': 'starmie', 'level': 100, 'types': ['psychic', 'water'], 'base_hp': 60, 'base_atk': 75, 'base_def': 85, 'base_spa': 100, 'base_spd': 100, 'base_spe': 115}, {'name': 'exeggutor', 'level': 100, 'types': ['grass', 'psychic'], 'base_hp': 95, 'base_atk': 95, 'base_def': 85, 'base_spa': 125, 'base_spd': 125, 'base_spe': 55}, {'name': 'chansey', 'level': 100, 'types': ['normal', 'notype'], 'base_hp': 250, 'base_atk': 5, 'base_def': 5, 'base_spa': 105, 'base_spd': 105, 'base_spe': 50}, {'name': 'snorlax', 'level': 100, 'types': ['normal', 'notype'], 'base_hp': 160, 'base_atk': 110, 'base_def': 65, 'base_spa': 65, 'base_spd': 65, 'base_spe': 30}, {'name': 'tauros', 'level': 100, 'types': ['normal', 'notype'], 'base_hp': 75, 'base_atk': 100, 'base_def': 95, 'base_spa': 70, 'base_spd': 70, 'base_spe': 110}, {'name': 'alakazam', 'level': 100, 'types': ['notype', 'psychic'], 'base_hp': 55, 'base_atk': 50, 'base_def': 45, 'base_spa': 135, 'base_spd': 135, 'base_spe': 120}, {'name': 'jynx', 

In [7]:
#Building second player team list

#the key must be 'p2_team_details'
def team2(battle):
  p2team=[]
  for turn in battle['battle_timeline']:

    name=turn['p2_pokemon_state']['name']

    for i in range(len(pokemon)):
      if pokemon[i]['name']==name:
        pok=pokemon[i]

    if pok not in p2team:
      p2team.append(pok)
  return p2team

for battle in train_data:
  battle['p2_team_details'] = team2(battle)
for battle in test_data:
  battle['p2_team_details'] = team2(battle)

print(train_data[1]['p2_team_details'])

[{'name': 'alakazam', 'level': 100, 'types': ['notype', 'psychic'], 'base_hp': 55, 'base_atk': 50, 'base_def': 45, 'base_spa': 135, 'base_spd': 135, 'base_spe': 120}, {'name': 'chansey', 'level': 100, 'types': ['normal', 'notype'], 'base_hp': 250, 'base_atk': 5, 'base_def': 5, 'base_spa': 105, 'base_spd': 105, 'base_spe': 50}, {'name': 'snorlax', 'level': 100, 'types': ['normal', 'notype'], 'base_hp': 160, 'base_atk': 110, 'base_def': 65, 'base_spa': 65, 'base_spd': 65, 'base_spe': 30}, {'name': 'exeggutor', 'level': 100, 'types': ['grass', 'psychic'], 'base_hp': 95, 'base_atk': 95, 'base_def': 85, 'base_spa': 125, 'base_spd': 125, 'base_spe': 55}, {'name': 'starmie', 'level': 100, 'types': ['psychic', 'water'], 'base_hp': 60, 'base_atk': 75, 'base_def': 85, 'base_spa': 100, 'base_spd': 100, 'base_spe': 115}, {'name': 'tauros', 'level': 100, 'types': ['normal', 'notype'], 'base_hp': 75, 'base_atk': 100, 'base_def': 95, 'base_spa': 70, 'base_spd': 70, 'base_spe': 110}]


In [8]:
df=pd.DataFrame(train_data)
df_test = pd.DataFrame(test_data)

def missing_values(df):
# Select the specified columns
  selected_columns = ['battle_timeline', 'p1_team_details', 'p2_team_details']
  df_check_missing = df[selected_columns]

# Check for missing values and count them
  missing_values = df_check_missing.isnull().sum()

# Print the missing value counts
  print("Missing values in essential columns:")
  print(missing_values)

missing_values(df)
missing_values(df_test)

Missing values in essential columns:
battle_timeline    0
p1_team_details    0
p2_team_details    0
dtype: int64
Missing values in essential columns:
battle_timeline    0
p1_team_details    0
p2_team_details    0
dtype: int64


In [9]:
#check team 1 size

# Iterate through each battle in the train_data list
def check_team1(battle):
    battle_id = battle.get('battle_id', 'N/A')  # Get battle_id, default to 'N/A' if not present

    # Access the team details
    p1_team = battle.get('p1_team_details', [])

    # Check the length of both lists
    if len(p1_team) != 6:
        print(f"Battle ID: {battle_id}, Player 1 team size is not 6: {len(p1_team)}")

for battle in train_data:
    check_team1(battle)
for battle in test_data:
    check_team1(battle)

In [10]:
def hp_errors(i, battle):
    battle_id = battle.get('battle_id', f'Index_{i}') # Use battle_id if available, otherwise index

    timeline = battle.get('battle_timeline', [])
    if not timeline:
        print(f"Warning: Battle ID {battle_id} has an empty battle_timeline.")
        return # Exit the function if timeline is empty

    # Check for sequential turn numbers
    expected_turn = 1
    for turn in timeline:
        current_turn = turn.get('turn')
        if current_turn is not None:
            if current_turn != expected_turn:
                 print(f"Warning: Battle ID {battle_id}, Non-sequential turn number: Expected {expected_turn}, got {current_turn}")
            expected_turn = current_turn + 1


        p1_state = turn.get('p1_pokemon_state', {})
        p2_state = turn.get('p2_pokemon_state', {})

        # Check HP percentages
        p1_hp_pct = p1_state.get('hp_pct')
        p2_hp_pct = p2_state.get('hp_pct')

        if p1_hp_pct is not None and (p1_hp_pct < 0.0 or p1_hp_pct > 1.0):
            print(f"Warning: Battle ID {battle_id}, Turn {current_turn}: Invalid P1 HP percentage: {p1_hp_pct}")

        if p2_hp_pct is not None and (p2_hp_pct < 0.0 or p2_hp_pct > 1.0):
            print(f"Warning: Battle ID {battle_id}, Turn {current_turn}: Invalid P2 HP percentage: {p2_hp_pct}")

        # Check move details base power if move is present
        p1_move = turn.get('p1_move_details')
        if p1_move is not None:
            p1_base_power = p1_move.get('base_power')
            if p1_base_power is not None and p1_base_power < 0:
                 print(f"Warning: Battle ID {battle_id}, Turn {current_turn}: Invalid P1 move base power: {p1_base_power}")

        p2_move = turn.get('p2_move_details')
        if p2_move is not None:
            p2_base_power = p2_move.get('base_power')
            if p2_base_power is not None and p2_base_power < 0:
                print(f"Warning: Battle ID {battle_id}, Turn {current_turn}: Invalid P2 move base power: {p2_base_power}")

for i, battle in enumerate(train_data):
    hp_errors(i, battle)
for i, battle in enumerate(test_data):
    hp_errors(i, battle)

In [11]:
# List of healing move names (based on general knowledge or can be inferred from data if needed)
healing_moves = ['recover', 'rest', 'softboiled']

for i, battle in enumerate(train_data):
    battle_id = battle.get('battle_id', f'Index_{i}')
    timeline = battle.get('battle_timeline', [])

    # Store the state of each pokemon in the team across turns
    p1_pokemon_states = {}
    p2_pokemon_states = {}

    for turn in timeline:
        current_turn_num = turn.get('turn')
        if current_turn_num is None:
            continue # Skip turns without a turn number

        p1_state = turn.get('p1_pokemon_state', {})
        p2_state = turn.get('p2_pokemon_state', {})

        # Update states for the active pokemon
        p1_name = p1_state.get('name')
        p2_name = p2_state.get('name')

        if p1_name:
            p1_pokemon_states[p1_name] = p1_state
        if p2_name:
            p2_pokemon_states[p2_name] = p2_state

        if current_turn_num > 1: # Check HP changes from the second turn onwards
            previous_turn_num = current_turn_num - 1
            previous_turn = timeline[previous_turn_num -1] # Access previous turn using index

            p1_previous_state = previous_turn.get('p1_pokemon_state', {})
            p2_previous_state = previous_turn.get('p2_pokemon_state', {})

            # Check Player 1's Pokémon HP
            p1_current_hp = p1_state.get('hp_pct', 0.0)
            # Get the previous HP of the *same* pokemon
            p1_previous_hp = p1_pokemon_states.get(p1_name, {}).get('hp_pct', 0.0) if p1_name else 0.0
            p1_move_details = turn.get('p1_move_details')
            p1_move_name = p1_move_details.get('name') if p1_move_details else None

            if p1_current_hp > p1_previous_hp and p1_move_name not in healing_moves:
                # print(f"Warning: Battle ID {battle_id}, Turn {current_turn_num}: P1 HP increased ({p1_previous_hp:.2f} -> {p1_current_hp:.2f}) without a recognized healing move ({p1_move_name}).")
                pass # Suppress printing for now, as we will filter later

            # Check Player 2's Pokémon HP
            p2_current_hp = p2_state.get('hp_pct', 0.0)
            # Get the previous HP of the *same* pokemon
            p2_previous_hp = p2_pokemon_states.get(p2_name, {}).get('hp_pct', 0.0) if p2_name else 0.0
            p2_move_details = turn.get('p2_move_details')
            p2_move_name = p2_move_details.get('name') if p2_move_details else None

            if p2_current_hp > p2_previous_hp and p2_move_name not in healing_moves:
                # print(f"Warning: Battle ID {battle_id}, Turn {current_turn_num}: P2 HP increased ({p2_previous_hp:.2f} -> {p2_current_hp:.2f}) without a recognized healing move ({p2_move_name}).")
                pass # Suppress printing for now, as we will filter later

# Feature Engeneering

In [12]:
#feature containing the sum of the stats of all pok
def total_stats_pokemon(battle):
  p1t=battle['p1_team_details']
  p2t=battle['p2_team_details']
  for pok in p1t:
    hp = pok['base_hp']
    atk = pok['base_atk']
    defense = pok['base_def']
    spatk = pok['base_spa']
    spdef = pok['base_spd']
    speed = pok['base_spe']
    total = hp + atk + defense + spatk + spdef + speed
    pok['total_stats'] = total
  for pok in p2t:
    hp = pok['base_hp']
    atk = pok['base_atk']
    defense = pok['base_def']
    spatk = pok['base_spa']
    spdef = pok['base_spd']
    speed = pok['base_spe']
    total = hp + atk + defense + spatk + spdef + speed
    pok['total_stats'] = total

  return p1t, p2t

for battle in train_data:
  battle['total_stats_p1'] = total_stats_pokemon(battle)
for battle in test_data:
  battle['total_stats_p2'] = total_stats_pokemon(battle)

print(train_data[1]['p1_team_details'])
print(train_data[1]['p2_team_details'])

[{'name': 'jynx', 'level': 100, 'types': ['ice', 'psychic'], 'base_hp': 65, 'base_atk': 50, 'base_def': 35, 'base_spa': 95, 'base_spd': 95, 'base_spe': 95, 'total_stats': 435}, {'name': 'snorlax', 'level': 100, 'types': ['normal', 'notype'], 'base_hp': 160, 'base_atk': 110, 'base_def': 65, 'base_spa': 65, 'base_spd': 65, 'base_spe': 30, 'total_stats': 495}, {'name': 'exeggutor', 'level': 100, 'types': ['grass', 'psychic'], 'base_hp': 95, 'base_atk': 95, 'base_def': 85, 'base_spa': 125, 'base_spd': 125, 'base_spe': 55, 'total_stats': 580}, {'name': 'tauros', 'level': 100, 'types': ['normal', 'notype'], 'base_hp': 75, 'base_atk': 100, 'base_def': 95, 'base_spa': 70, 'base_spd': 70, 'base_spe': 110, 'total_stats': 520}, {'name': 'chansey', 'level': 100, 'types': ['normal', 'notype'], 'base_hp': 250, 'base_atk': 5, 'base_def': 5, 'base_spa': 105, 'base_spd': 105, 'base_spe': 50, 'total_stats': 520}, {'name': 'slowbro', 'level': 100, 'types': ['psychic', 'water'], 'base_hp': 95, 'base_atk':

In [13]:
# creating a list of all the types

types = []
for pok in pokemon:
  if pok['types'][0] not in types:
    types.append(pok['types'][0])
  if pok['types'][1] not in types:
    types.append(pok['types'][1])
print(types)

['psychic', 'water', 'grass', 'normal', 'notype', 'ice', 'ghost', 'poison', 'ground', 'rock', 'electric', 'flying', 'dragon', 'fire']


In [14]:
#list of moves
all_moves = []
for battle in train_data:
  for turn in battle['battle_timeline']:
    if turn['p1_move_details'] is not None:
        move_name=turn['p1_move_details']['name']
        all_moves.append(move_name)
    if turn['p2_move_details'] is not None:
        move_name=turn['p2_move_details']['name']
        all_moves.append(move_name)

# Get unique moves
unique_moves = list(set(all_moves))
print(unique_moves)
print(len(unique_moves))

['icebeam', 'slash', 'sing', 'psychic', 'rest', 'softboiled', 'clamp', 'earthquake', 'toxic', 'amnesia', 'swordsdance', 'nightshade', 'thunderbolt', 'doublekick', 'selfdestruct', 'hypnosis', 'recover', 'explosion', 'thunderwave', 'reflect', 'razorleaf', 'sleeppowder', 'fireblast', 'blizzard', 'stunspore', 'seismictoss', 'surf', 'wrap', 'agility', 'firespin', 'bodyslam', 'substitute', 'doubleedge', 'confuseray', 'lovelykiss', 'hyperbeam', 'megadrain', 'counter', 'drillpeck', 'rockslide']
40


In [15]:
# recovering moves
recovering_moves = []

for battle in train_data:
  for i, turn in enumerate(battle['battle_timeline']):
    if i == 0:
      continue
    prev_turn = battle['battle_timeline'][i-1]
    pok1 = turn['p1_pokemon_state']['name']
    pok2 = turn['p2_pokemon_state']['name']
    if pok1 == prev_turn['p1_pokemon_state']['name']:
      life_before1 = prev_turn['p1_pokemon_state']['hp_pct']
      life_after1 = turn['p1_pokemon_state']['hp_pct']
      if life_before1 < life_after1:
        if turn['p1_move_details'] is not None:
          move_name = turn['p1_move_details']['name']
          if move_name not in recovering_moves:
            recovering_moves.append(move_name)

    if pok2 == prev_turn['p2_pokemon_state']['name']:
      life_before2 = prev_turn['p2_pokemon_state']['hp_pct']
      life_after2 = turn['p2_pokemon_state']['hp_pct']
      if life_before2 < life_after2:
        if turn['p2_move_details'] is not None:
          move_name = turn['p2_move_details']['name']
          if move_name not in recovering_moves:
            recovering_moves.append(move_name)

print(recovering_moves)

['recover', 'rest', 'softboiled', 'megadrain']


In [16]:
#dividing damaging moves form non damaging ones
damaging_moves = []
move_details = {}
for battle in train_data:
    for turn in battle['battle_timeline']:
        if turn['p1_move_details'] is not None:
            move = turn['p1_move_details']
            if move['name'] not in move_details:
                move_details[move['name']] = move
        if turn['p2_move_details'] is not None:
            move = turn['p2_move_details']
            if move['name'] not in move_details:
                move_details[move['name']] = move


for move_name, details in move_details.items():
    if details['base_power'] > 0:
        damaging_moves.append(move_name)
print(damaging_moves)
print(len(damaging_moves))

['icebeam', 'blizzard', 'thunderbolt', 'bodyslam', 'psychic', 'explosion', 'seismictoss', 'doubleedge', 'counter', 'selfdestruct', 'hyperbeam', 'fireblast', 'earthquake', 'razorleaf', 'megadrain', 'drillpeck', 'slash', 'surf', 'nightshade', 'wrap', 'clamp', 'rockslide', 'doublekick', 'firespin']
24


In [17]:
# ensuring priority is only equal 0 or -1
for battle in train_data:
  for turn in battle['battle_timeline']:
    if turn['p2_move_details'] is not None:
      if turn['p2_move_details']['priority'] != 0 and turn['p2_move_details']['priority'] != -1:
        print(turn['p2_move_details']['priority'])
    if turn['p1_move_details'] is not None:
      if turn['p1_move_details']['priority'] != 0 and turn['p1_move_details']['priority'] != -1:
        print(turn['p1_move_details']['priority'])

In [18]:
# priority

def priority(battle):
  priority1 = 0
  priority2 = 0
  for turn in battle['battle_timeline']:
    if turn['p1_move_details'] != None:
      if turn['p1_move_details']['priority'] == -1:
        priority1 -= 1
    if turn['p2_move_details'] != None:
      if turn['p2_move_details']['priority'] == -1:
        priority2 -= 1
  return priority1, priority2, priority1 - priority2

for battle in train_data:
  battle['priority1']=priority(battle)[0]
  battle['priority2']=priority(battle)[1]
  battle['priority_diff'] = priority(battle)[2]

for battle in test_data:
  battle['priority1']=priority(battle)[0]
  battle['priority2']=priority(battle)[1]
  battle['priority_diff'] = priority(battle)[2]


df['priority_diff'] = [battle['priority_diff'] for battle in train_data]
df['priority1']=[battle['priority1'] for battle in train_data]
df['priority2']=[battle['priority2'] for battle in train_data]

df_test['priority2']=[battle['priority2'] for battle in test_data]
df_test['priority1'] = [battle['priority1'] for battle in test_data]
df_test['priority_diff'] = [battle['priority_diff'] for battle in test_data]

In [30]:
def special(battle):
  special1 = 0
  special2 = 0
  for turn in battle['battle_timeline']:
    if turn['p1_move_details'] != None:
      if turn['p1_move_details']['category'] == 'SPECIAL':
        special1 += 1
    if turn['p2_move_details'] != None:
      if turn['p2_move_details']['category'] == 'SPECIAL':
        special2 += 1
  return special1, special2, special1 - special2

for battle in train_data:
  battle['special_diff'] = special(battle)[2]
  battle['special1']=special(battle)[0]
  battle['special2']=special(battle)[1]

for battle in test_data:
  battle['special_diff'] = special(battle)[2]
  battle['special1']=special(battle)[0]
  battle['special2']=special(battle)[1]


df['special_diff'] = [battle['special_diff'] for battle in train_data]
df['special1']=[battle['special1'] for battle in train_data]
df['special2']=[battle['special2'] for battle in train_data]

df_test['special_diff'] = [battle['special_diff'] for battle in test_data]
df_test['special1']=[battle['special1'] for battle in test_data]
df_test['special2']=[battle['special2'] for battle in test_data]

In [20]:
# Create a list containing info about types and their multiplier vs all other types

pokemon_by_name = {p['name']: p for p in pokemon}
typess = [t for t in types if t not in ["normal", "notype"]]
typesss = [t for t in types if t not in ["notype"]]

type_inf = []

for typee in typesss:
    total_damage = 0
    total_moves = 0

    vs_stats = {t: {"damage": 0, "moves": 0} for t in typess}

    for battle in train_data:
        timeline = battle['battle_timeline']
        for i in range(1, len(timeline)):
            turn = timeline[i]
            prev_turn = timeline[i - 1]

            pok1 = turn['p1_pokemon_state']['name']
            pok2 = turn['p2_pokemon_state']['name']
            pok1_data = pokemon_by_name[pok1]
            pok2_data = pokemon_by_name[pok2]

            if typee in pok1_data['types'] and (pok2_data['types'] in typess or pok2_data['types'][0] in typess):
                if (pok2 == prev_turn['p2_pokemon_state']['name'] and
                    pok1 == prev_turn['p1_pokemon_state']['name']):
                  if turn['p1_move_details'] != None and turn['p2_move_details'] != None:
                      if (turn['p2_move_details']['name'] not in recovering_moves and
                          turn['p1_move_details']['name'] in damaging_moves):

                          damage = (prev_turn['p2_pokemon_state']['hp_pct'] - turn['p2_pokemon_state']['hp_pct']) * pok2_data['base_hp']
                          total_damage += damage
                          total_moves += 1
                          for t in pok2_data['types']:
                            if t != 'normal' and t != 'notype':
                              vs_stats[t]["damage"] += damage
                              vs_stats[t]["moves"] += 1

            if typee in pok2_data['types'] and (pok1_data['types'][0] in typess or pok1_data['types'][1] in typess):
                if (pok1 == prev_turn['p1_pokemon_state']['name'] and
                    pok2 == prev_turn['p2_pokemon_state']['name']):
                  if turn['p1_move_details'] != None and turn['p2_move_details'] != None:
                      if (turn['p1_move_details']['name'] not in recovering_moves and
                        turn['p2_move_details']['name'] in damaging_moves):

                        damage = (prev_turn['p1_pokemon_state']['hp_pct'] - turn['p1_pokemon_state']['hp_pct']) * pok1_data['base_hp']
                        total_damage += damage
                        total_moves += 1
                        for t in pok1_data['types']:
                          if t != 'normal' and t != 'notype':
                              vs_stats[t]["damage"] += damage
                              vs_stats[t]["moves"] += 1

    avg_damage = total_damage / total_moves if total_moves > 0 else 0
    type_info = {
        "type_name": typee,
        "total_damage": total_damage,
        "total_moves": total_moves,
        "avg_damage": avg_damage}

    for t, stats in vs_stats.items():
      if stats["moves"] > 0 and stats["damage"] > 0:
        avg_vs = stats["damage"] / stats["moves"]
        mult = avg_vs / avg_damage
      else:
        mult = 1

      type_info[f"vs_{t}"] = mult

    type_inf.append(type_info)

In [21]:
# Creating multipliers vs type = normal

pokemon_by_name = {p['name']: p for p in pokemon}
typess = [t for t in types if t not in ["notype"]]

for typee in typess:
    total_damage = 0
    total_moves = 0

    vs_stats = {t: {"damage": 0, "moves": 0} for t in ["normal"]}

    for battle in train_data:
        timeline = battle['battle_timeline']
        for i in range(1, len(timeline)):
            turn = timeline[i]
            prev_turn = timeline[i - 1]

            pok1 = turn['p1_pokemon_state']['name']
            pok2 = turn['p2_pokemon_state']['name']
            pok1_data = pokemon_by_name[pok1]
            pok2_data = pokemon_by_name[pok2]

            if typee in pok1_data['types'] :
                if (pok2 == prev_turn['p2_pokemon_state']['name'] and
                    pok1 == prev_turn['p1_pokemon_state']['name']):
                  if turn['p1_move_details'] != None and turn['p2_move_details'] != None:
                      if (turn['p2_move_details']['name'] not in recovering_moves and
                          turn['p1_move_details']['name'] in damaging_moves):

                          damage = (prev_turn['p2_pokemon_state']['hp_pct'] - turn['p2_pokemon_state']['hp_pct']) * pok2_data['base_hp']
                          total_damage += damage
                          total_moves += 1
                          for t in pok2_data['types']:
                            if t == 'normal':
                              vs_stats[t]["damage"] += damage
                              vs_stats[t]["moves"] += 1

            if typee in pok2_data['types'] :
                if (pok1 == prev_turn['p1_pokemon_state']['name'] and
                    pok2 == prev_turn['p2_pokemon_state']['name']):
                  if turn['p1_move_details'] != None and turn['p2_move_details'] != None:
                      if (turn['p1_move_details']['name'] not in recovering_moves and
                        turn['p2_move_details']['name'] in damaging_moves):

                        damage = (prev_turn['p1_pokemon_state']['hp_pct'] - turn['p1_pokemon_state']['hp_pct']) * pok1_data['base_hp']
                        total_damage += damage
                        total_moves += 1
                        for t in pok1_data['types']:
                          if t == 'normal':
                              vs_stats[t]["damage"] += damage
                              vs_stats[t]["moves"] += 1

    avg_damage = total_damage / total_moves if total_moves > 0 else 0

    for t, stats in vs_stats.items():
      if stats["moves"] > 0 and stats["damage"] > 0:
        avg_vs = stats["damage"] / stats["moves"]
        mult = avg_vs / avg_damage
      else:
        mult = 1
      for i in range(len(type_inf)):
        if type_inf[i]['type_name'] == typee:
          type_inf[i]['vs_normal'] = mult

In [22]:
# Feature to capture the advantage of the first player based on the pokemons types

type_chart = {t['type_name']: t for t in type_inf}

def type_advantage(battle):
    battle['types_adv'] = 0

    for pok1 in battle['p1_team_details']:
        for type1 in pok1['types']:
            for pok2 in battle['p2_team_details']:
                for type2 in pok2['types']:
                    if type1 == 'notype' or type2 == 'notype':
                        continue

                    battle['types_adv'] += type_chart[type1][f'vs_{type2}']
                    battle['types_adv'] -= type_chart[type2][f'vs_{type1}']

for battle in train_data:
  type_advantage(battle)
for battle in test_data:
  type_advantage(battle)

df['type_adv'] = [battle['types_adv'] for battle in train_data]
df_test['type_adv'] = [battle['types_adv'] for battle in test_data]

In [23]:
multipliers = {
    "normal": {"rock": 0.5, "ghost": 0.0, "steel": 0.5},
    "fire": {"fire": 0.5, "water": 0.5, "grass": 2.0, "ice": 2.0, "bug": 2.0, "rock": 0.5, "dragon": 0.5, "steel": 2.0},
    "water": {"fire": 2.0, "water": 0.5, "grass": 0.5, "ground": 2.0, "rock": 2.0, "dragon": 0.5},
    "electric": {"water": 2.0, "electric": 0.5, "grass": 0.5, "ground": 0.0, "flying": 2.0, "dragon": 0.5},
    "grass": {"fire": 0.5, "water": 2.0, "grass": 0.5, "poison": 0.5, "ground": 2.0, "flying": 0.5, "bug": 0.5, "rock": 2.0, "dragon": 0.5, "steel": 0.5},
    "ice": {"fire": 0.5, "water": 0.5, "grass": 2.0, "ground": 2.0, "flying": 2.0, "dragon": 2.0, "steel": 0.5},
    "fighting": {"normal": 2.0, "ice": 2.0, "poison": 0.5, "flying": 0.5, "psychic": 0.5, "bug": 0.5, "rock": 2.0, "ghost": 0.0, "dark": 2.0, "steel": 2.0, "fairy": 0.5},
    "poison": {"grass": 2.0, "poison": 0.5, "ground": 0.5, "rock": 0.5, "ghost": 0.5, "steel": 0.0, "fairy": 2.0},
    "ground": {"fire": 2.0, "electric": 2.0, "grass": 0.5, "poison": 2.0, "flying": 0.0, "bug": 0.5, "rock": 2.0, "steel": 2.0},
    "flying": {"electric": 0.5, "grass": 2.0, "fighting": 2.0, "bug": 2.0, "rock": 0.5, "steel": 0.5},
    "psychic": {"fighting": 2.0, "poison": 2.0, "psychic": 0.5, "dark": 0.0, "steel": 0.5},
    "bug": {"fire": 0.5, "grass": 2.0, "fighting": 0.5, "poison": 0.5, "flying": 0.5, "psychic": 2.0, "ghost": 0.5, "dark": 2.0, "steel": 0.5, "fairy": 0.5},
    "rock": {"fire": 2.0, "ice": 2.0, "fighting": 0.5, "ground": 0.5, "flying": 2.0, "bug": 2.0, "steel": 0.5},
    "ghost": {"normal": 0.0, "psychic": 2.0, "ghost": 2.0, "dark": 0.5},
    "dragon": {"dragon": 2.0, "steel": 0.5, "fairy": 0.0},
    "dark": {"fighting": 0.5, "psychic": 2.0, "ghost": 2.0, "dark": 0.5, "fairy": 0.5},
    "steel": {"fire": 0.5, "water": 0.5, "electric": 0.5, "ice": 2.0, "rock": 2.0, "steel": 0.5, "fairy": 2.0},
    "fairy": {"fire": 0.5, "fighting": 2.0, "poison": 0.5, "dragon": 2.0, "dark": 2.0, "steel": 0.5}}


In [24]:
def get_multiplier(move_type, defender_types):

    multiplier = 1.0
    for d_type in defender_types:
        multiplier *= multipliers.get(move_type, {}).get(d_type, 1.0)
    return multiplier

In [25]:
def multiplier(battle):
  for turn in battle['battle_timeline']:
    if turn['p1_move_details'] != None:
      move_type1 = turn['p1_move_details']['type'].lower()
      defender_name1 = turn['p2_pokemon_state']['name']

      defender_pokemon1 = pokemon_by_name[defender_name1]
      defender_types1 = defender_pokemon1.get('types', [])

      turn['p1_move_details']['type_multiplier'] = get_multiplier(move_type1, defender_types1)

    if turn['p2_move_details'] != None:
      move_type2 = turn['p2_move_details']['type'].lower()
      defender_name2 = turn['p1_pokemon_state']['name']

      defender_pokemon2 = pokemon_by_name[defender_name2]
      defender_types2 = defender_pokemon2.get('types', [])

      turn['p2_move_details']['type_multiplier'] = get_multiplier(move_type2, defender_types2)

for battle in train_data:
  multiplier(battle)
for battle in test_data:
  multiplier(battle)

In [29]:
def mult_diff(battle):
  mult1 = 0
  mult2 = 0
  for turn in battle['battle_timeline']:
    if turn['p1_move_details'] != None:
      if turn['p1_move_details']['name'] in damaging_moves:
        mult1 += turn['p1_move_details']['type_multiplier']
    if turn['p2_move_details'] != None:
      if turn['p2_move_details']['name'] in damaging_moves:
        mult2 += turn['p2_move_details']['type_multiplier']
  battle['mult1']=mult1
  battle['mult2']=mult2
  battle['mult_diff'] = mult1 - mult2

for battle in train_data:
  mult_diff(battle)

for battle in test_data:
  mult_diff(battle)

df['mult_diff'] = [battle['mult_diff'] for battle in train_data]
df['mult1']=[battle['mult1'] for battle in train_data]
df['mult2']=[battle['mult2'] for battle in train_data]

df_test['mult_diff'] = [battle['mult_diff'] for battle in test_data]
df_test['mult1']=[battle['mult1'] for battle in test_data]
df_test['mult2']=[battle['mult2'] for battle in test_data]

In [28]:
def damage_diff(battle):

  damage1 = 0
  damage2 = 0

  timeline = battle.get('battle_timeline', [])
  if not timeline:
      return 0 # Return 0 if timeline is empty

  for i in range(1, len(timeline)):
    turn = timeline[i]
    prev_turn = timeline[i - 1]

    # Get pokemon names, handling potential missing keys
    pok1_name = turn.get('p1_pokemon_state', {}).get('name')
    pok2_name = turn.get('p2_pokemon_state', {}).get('name')

    if not pok1_name or not pok2_name:
        continue # Skip turn if pokemon names are missing

    # Get pokemon data, handling potential missing pokemon in dictionary
    pok1_data = pokemon_by_name.get(pok1_name)
    pok2_data = pokemon_by_name.get(pok2_name)

    if not pok1_data or not pok2_data:
        continue # Skip turn if pokemon data is missing

    # Get previous pokemon state, handling potential missing keys
    prev_p1_state = prev_turn.get('p1_pokemon_state', {})
    prev_p2_state = prev_turn.get('p2_pokemon_state', {})

    # Check if the same pokemon are in battle as the previous turn
    if (pok2_name == prev_p2_state.get('name') and pok1_name == prev_p1_state.get('name')):

      # Get move details, handling potential missing keys
      p1_move_details = turn.get('p1_move_details')
      p2_move_details = turn.get('p2_move_details')

      if p1_move_details is not None and p2_move_details is not None:
        p1_move_name = p1_move_details.get('name')
        p2_move_name = p2_move_details.get('name')

        # Check if moves are damaging and not healing, handling potential missing move names
        if p1_move_name in damaging_moves and p2_move_name not in recovering_moves:
          prev_p2_hp_pct = prev_p2_state.get('hp_pct')
          curr_p2_hp_pct = turn.get('p2_pokemon_state', {}).get('hp_pct')
          pok2_base_hp = pok2_data.get('base_hp')

          if prev_p2_hp_pct is not None and curr_p2_hp_pct is not None and pok2_base_hp is not None:
              damage1 += (prev_p2_hp_pct - curr_p2_hp_pct) * pok2_base_hp


        if p2_move_name in damaging_moves and p1_move_name not in recovering_moves:
          prev_p1_hp_pct = prev_p1_state.get('hp_pct')
          curr_p1_hp_pct = turn.get('p1_pokemon_state', {}).get('hp_pct')
          pok1_base_hp = pok1_data.get('base_hp')

          if prev_p1_hp_pct is not None and curr_p1_hp_pct is not None and pok1_base_hp is not None:
              damage2 += (prev_p1_hp_pct - curr_p1_hp_pct) * pok1_base_hp

  damage_diff_value = damage1 - damage2
  return damage1, damage2, damage_diff_value


for battle in train_data:
  battle['damage1'] = damage_diff(battle)[0]
  battle['damage2'] = damage_diff(battle)[1]
  battle['damage_diff'] = damage_diff(battle)[2]

for battle in test_data:
  battle['damage1'] = damage_diff(battle)[0]
  battle['damage2'] = damage_diff(battle)[1]
  battle['damage_diff'] = damage_diff(battle)[2]

df_test['damage1'] = [battle['damage1'] for battle in test_data]
df_test['damage2'] = [battle['damage2'] for battle in test_data]
df_test['damage_diff'] = [battle['damage_diff'] for battle in test_data]

df['damage1'] = [battle['damage1'] for battle in train_data]
df['damage2'] = [battle['damage2'] for battle in train_data]
df['damage_diff'] = [battle['damage_diff'] for battle in train_data]

In [31]:
def hyper_beam_ko_flags(timeline):
    output = {"p1_hb_ko":0.0, "p2_hb_ko":0.0}
    for turn in timeline or []:
        move_p1 = (turn.get("p1_move_details") or {})
        move_p2 = (turn.get("p2_move_details") or {})
        if str(move_p1.get("name","")).lower()=="hyperbeam" and (turn.get("p2_pokemon_state") or {}).get("status")=="fnt":
            output["p1_hb_ko"]=1.0
        if str(move_p2.get("name","")).lower()=="hyperbeam" and (turn.get("p1_pokemon_state") or {}).get("status")=="fnt":
            output["p2_hb_ko"]=1.0
    return output

In [32]:
# Apply the hyper_beam_ko_flags function to the dataframes
df[['p1_hb_ko', 'p2_hb_ko']] = df['battle_timeline'].apply(lambda x: pd.Series(hyper_beam_ko_flags(x)))
df_test[['p1_hb_ko', 'p2_hb_ko']] = df_test['battle_timeline'].apply(lambda x: pd.Series(hyper_beam_ko_flags(x)))

In [33]:
def early_cc_features(timeline, window_size=5):
    features = {"first_cc_winner":0.0, "early_sleep_hits":0.0, "early_freeze_hits":0.0}
    first_crowd_control = None
    for turn_index, turn in enumerate(timeline[:window_size] or [], start=1):

        status_p1 = (turn.get("p1_pokemon_state") or {}).get("status")
        status_p2 = (turn.get("p2_pokemon_state") or {}).get("status")

        if first_crowd_control is None:
            if status_p1 in ("slp","frz") and status_p2 not in ("slp","frz"):
                first_crowd_control = -1
            if status_p2 in ("slp","frz") and status_p1 not in ("slp","frz"):
                first_crowd_control = +1

        if status_p2 == "slp":
            features["early_sleep_hits"] += 1
        if status_p1 == "slp":
            features["early_sleep_hits"] -= 1
        if status_p2 == "frz":
            features["early_freeze_hits"] += 1
        if status_p1 == "frz":
            features["early_freeze_hits"] -= 1

    features["first_cc_winner"] = float(first_crowd_control or 0)
    return features


df[['first_cc_winner', 'early_sleep_hits', 'early_freeze_hits']] = df['battle_timeline'].apply(lambda x: pd.Series(early_cc_features(x)))
df_test[['first_cc_winner', 'early_sleep_hits', 'early_freeze_hits']] = df_test['battle_timeline'].apply(lambda x: pd.Series(early_cc_features(x)))

In [34]:
def stab_adjusted_bp(move, attacker_name: str) -> float:
    # Questa funzione calcola la potenza base di una mossa, aggiustata per lo STAB (Same Type Attack Bonus).
    # Lo STAB si applica se il tipo della mossa corrisponde a uno dei tipi del Pokémon attaccante,
    # conferendo un bonus del 50% alla potenza della mossa.

    if not move:
        return 0.0

    # Estrae la potenza base della mossa. Se non è presente o non è un numero valido, viene considerato 0.
    base_power = float(move.get("base_power") or 0)
    if base_power <= 0:
        return 0.0

    # Estrae il tipo della mossa e lo converte in minuscolo.
    move_type = str(move.get("type") or "").lower()
    # Se il tipo della mossa o il nome dell'attaccante non sono validi, ritorna la potenza base senza modifiche.
    if not move_type or not attacker_name:
        return base_power

    # Cerca i tipi del Pokémon attaccante nel dizionario NAME_TO_TYPES.
    attacker_types = NAME_TO_TYPES.get(attacker_name, [])

    # Applica il bonus STAB (moltiplica per 1.5) se il tipo della mossa è presente tra i tipi dell'attaccante,
    # altrimenti ritorna la potenza base originale.
    return base_power * 1.5 if move_type in attacker_types else base_power

In [35]:
NAME_TO_TYPES = {p['name']: p['types'] for p in pokemon}

def total_stab_adjusted_bp(battle):

    p1_total_stab_bp = 0.0
    p2_total_stab_bp = 0.0

    for turn in battle.get('battle_timeline', []):
        p1_move = turn.get('p1_move_details')
        p2_move = turn.get('p2_move_details')
        p1_pokemon_name = turn.get('p1_pokemon_state', {}).get('name')
        p2_pokemon_name = turn.get('p2_pokemon_state', {}).get('name')

        if p1_move and p1_pokemon_name:
            p1_total_stab_bp += stab_adjusted_bp(p1_move, p1_pokemon_name)

        if p2_move and p2_pokemon_name:
            p2_total_stab_bp += stab_adjusted_bp(p2_move, p2_pokemon_name)

    return p1_total_stab_bp, p2_total_stab_bp

# Apply the function to the DataFrames and create new columns
df[['p1_total_stab_bp', 'p2_total_stab_bp']] = df.apply(
    total_stab_adjusted_bp, axis=1, result_type='expand')

df_test[['p1_total_stab_bp', 'p2_total_stab_bp']] = df_test.apply(
    total_stab_adjusted_bp, axis=1, result_type='expand')


# Create a difference feature
df['diff_total_stab_bp'] = df['p1_total_stab_bp'] - df['p2_total_stab_bp']
df_test['diff_total_stab_bp'] = df_test['p1_total_stab_bp'] - df_test['p2_total_stab_bp']

In [37]:
#boost count
def boost_count_computation(battle):
  tot_boostp1=0
  tot_boostp2=0

  for turn in battle.get('battle_timeline', []):
    # Get the pokemon state for player 1 and player 2 for the current turn
    p1_state = turn.get('p1_pokemon_state', {})
    p2_state = turn.get('p2_pokemon_state', {})

    # Count boosts for player 1's active pokemon in this turn
    if 'boosts' in p1_state:
      for boost_value in p1_state['boosts'].values():
        if boost_value > 0:
          tot_boostp1 += 1

    # Count boosts for player 2's active pokemon in this turn
    if 'boosts' in p2_state:
      for boost_value in p2_state['boosts'].values():
        if boost_value > 0:
          tot_boostp2 += 1
    return tot_boostp1, tot_boostp2

#applying the function to train andd test data
df[['tot_boostp1', 'tot_boostp2']] = df.apply(boost_count_computation, axis=1, result_type='expand')
df['diff_boosts']=df['tot_boostp1']-df['tot_boostp2']

df_test[['tot_boostp1', 'tot_boostp2']] = df_test.apply(boost_count_computation, axis=1, result_type='expand')
df_test['diff_boosts']=df_test['tot_boostp1']-df_test['tot_boostp2']

In [39]:
def compute_average_move_accuracy(battle):

    timeline = battle.get('battle_timeline', [])
    if not timeline:
        return 0.0, 0.0
    p1_accuracies = []
    p2_accuracies = []

    for turn in timeline:
        p1_move = turn.get('p1_move_details')
        p2_move = turn.get('p2_move_details')

        if p1_move and p1_move.get('accuracy') is not None:
            p1_accuracies.append(p1_move['accuracy'])

        if p2_move and p2_move.get('accuracy') is not None:
            p2_accuracies.append(p2_move['accuracy'])

    avg_accuracy_p1 = sum(p1_accuracies) / len(p1_accuracies) if p1_accuracies else 0.0
    avg_accuracy_p2 = sum(p2_accuracies) / len(p2_accuracies) if p2_accuracies else 0.0

    return avg_accuracy_p1, avg_accuracy_p2

df[['p1_avg_move_accuracy', 'p2_avg_move_accuracy']] = df.apply(
    compute_average_move_accuracy, axis=1, result_type='expand')
df['diff_move_accuracy'] = df['p1_avg_move_accuracy'] - df['p2_avg_move_accuracy']

df_test[['p1_avg_move_accuracy', 'p2_avg_move_accuracy']] = df_test.apply(
    compute_average_move_accuracy, axis=1, result_type='expand')
df_test['diff_move_accuracy'] = df_test['p1_avg_move_accuracy'] - df_test['p2_avg_move_accuracy']

In [41]:
def status_count_computation(battle):
  tot_statusp1 = 0
  tot_statusp2 = 0

  for turn in battle.get('battle_timeline', []):

    p1_state = turn.get('p1_pokemon_state', {})
    p2_state = turn.get('p2_pokemon_state', {})

    # Check status for player 1's active pokemon in this turn
    if 'status' in p1_state:
      # Directly check the status string
      if p1_state['status'] != 'nostatus':
        tot_statusp1 += 1

    # Check status for player 2's active pokemon in this turn
    if 'status' in p2_state:
      # Directly check the status string
      if p2_state['status'] != 'nostatus':
        tot_statusp2 += 1

  return tot_statusp1, tot_statusp2

df[['tot_statusp1', 'tot_statusp2']] = df.apply(status_count_computation, axis=1, result_type='expand')
df['diff_status'] = df['tot_statusp2'] - df['tot_statusp1']

df_test[['tot_statusp1', 'tot_statusp2']] = df_test.apply(status_count_computation, axis=1, result_type='expand')
df_test['diff_status'] = df_test['tot_statusp2'] - df_test['tot_statusp1']

In [44]:
def count_fainted_pokemon(battle):
   p1_fainted=set()
   p2_fainted=set()
   for turn in battle['battle_timeline']:
    p1_state=turn['p1_pokemon_state']
    p2_state=turn['p2_pokemon_state']
    if p1_state['hp_pct']==0.0:
      p1_fainted.add(p1_state['name'])
    if p2_state['hp_pct']==0.0:
      p2_fainted.add(p2_state['name'])
   return len(p1_fainted), len(p2_fainted)

df[['p1_fainted_pokemon', 'p2_fainted_pokemon']] = df.apply(
    count_fainted_pokemon, axis=1, result_type='expand')
df['diff_fainted_pokemon'] = df['p1_fainted_pokemon'] - df['p2_fainted_pokemon']

df_test[['p1_fainted_pokemon', 'p2_fainted_pokemon']] = df_test.apply(
    count_fainted_pokemon, axis=1, result_type='expand')
df_test['diff_fainted_pokemon'] = df_test['p1_fainted_pokemon'] - df_test['p2_fainted_pokemon']

In [46]:
def count_switches(battle):

    timeline = battle.get('battle_timeline', [])
    if not timeline or len(timeline) < 2:
        return 0, 0, 0

    p1_voluntary_switches = 0
    p2_voluntary_switches = 0

    p1_prev_pokemon_name = timeline[0].get('p1_pokemon_state', {}).get('name')
    p1_prev_pokemon_hp = timeline[0].get('p1_pokemon_state', {}).get('hp_pct', 1.0)

    p2_prev_pokemon_name = timeline[0].get('p2_pokemon_state', {}).get('name')
    p2_prev_pokemon_hp = timeline[0].get('p2_pokemon_state', {}).get('hp_pct', 1.0)

    for i in range(1, len(timeline)):
        turn = timeline[i]
        p1_state = turn.get('p1_pokemon_state', {})
        p2_state = turn.get('p2_pokemon_state', {})

        p1_current_pokemon_name = p1_state.get('name')
        p1_current_pokemon_hp = p1_state.get('hp_pct', 1.0)

        p2_current_pokemon_name = p2_state.get('name')
        p2_current_pokemon_hp = p2_state.get('hp_pct', 1.0)

        if p1_current_pokemon_name and p1_prev_pokemon_name and \
           p1_current_pokemon_name != p1_prev_pokemon_name and \
           p1_prev_pokemon_hp > 0:
            p1_voluntary_switches += 1

        if p2_current_pokemon_name and p2_prev_pokemon_name and \
           p2_current_pokemon_name != p2_prev_pokemon_name and \
           p2_prev_pokemon_hp > 0:
            p2_voluntary_switches += 1

        p1_prev_pokemon_name = p1_current_pokemon_name
        p1_prev_pokemon_hp = p1_current_pokemon_hp

        p2_prev_pokemon_name = p2_current_pokemon_name
        p2_prev_pokemon_hp = p2_current_pokemon_hp

    diff_switches = p1_voluntary_switches - p2_voluntary_switches

    return p1_voluntary_switches , p2_voluntary_switches , diff_switches

for battle in train_data:
  battle['p1_voluntary_switches'] = count_switches(battle)[0]
  battle['p2_voluntary_switches'] = count_switches(battle)[1]
  battle['diff_switches'] = count_switches(battle)[2]

for battle in test_data:
  battle['p1_voluntary_switches'] = count_switches(battle)[0]
  battle['p2_voluntary_switches'] = count_switches(battle)[1]
  battle['diff_switches'] = count_switches(battle)[2]

df['p1_voluntary_switches'] = [battle['p1_voluntary_switches'] for battle in train_data]
df['p2_voluntary_switches'] = [battle['p2_voluntary_switches'] for battle in train_data]
df['diff_switches'] = [battle['diff_switches'] for battle in train_data]

df_test['p1_voluntary_switches'] = [battle['p1_voluntary_switches'] for battle in test_data]
df_test['p2_voluntary_switches'] = [battle['p2_voluntary_switches'] for battle in test_data]
df_test['diff_switches'] = [battle['diff_switches'] for battle in test_data]

In [47]:
def count_healing_moves(battle):
    """
    Counts the number of healing moves used by each player in a battle.
    """
    timeline = battle.get('battle_timeline', [])
    if not timeline:
        return 0, 0

    p1_healing_moves = 0
    p2_healing_moves = 0

    healing_moves = ['recover', 'rest', 'softboiled', 'megadrain'] # Assuming these are the healing moves based on your previous code

    for turn in timeline:
        p1_move = turn.get('p1_move_details')
        p2_move = turn.get('p2_move_details')

        if p1_move and p1_move.get('name') in healing_moves:
            p1_healing_moves += 1

        if p2_move and p2_move.get('name') in healing_moves:
            p2_healing_moves += 1

    return p1_healing_moves, p2_healing_moves

# Apply the function to the DataFrame and create new columns
df[['p1_healing_moves', 'p2_healing_moves']] = df.apply(
    count_healing_moves, axis=1, result_type='expand')
df['diff_healing_moves'] = df['p1_healing_moves'] - df['p2_healing_moves']

df_test[['p1_healing_moves', 'p2_healing_moves']] = df_test.apply(
    count_healing_moves, axis=1, result_type='expand')
df_test['diff_healing_moves'] = df_test['p1_healing_moves'] - df_test['p2_healing_moves']

In [48]:
# Estimated life difference for the teams at turn 30

pokemon_by_name = {p['name']: p for p in pokemon}

def calculate_life_diff_turn30(battle):
    team1 = battle['p1_team_details']
    team2 = battle['p2_team_details']
    lifes1_at_turn_30 = []
    lifes2_at_turn_30 = []

    # Find the state of each pokemon in team1 at turn 30 or the last available turn before 30
    for pok in team1:
        pok1_data = pokemon_by_name[pok['name']]
        life = pok1_data['base_hp']
        last_available_state = None

        for turn in battle['battle_timeline']:
            if turn.get('turn') is not None and turn['turn'] <= 30:
                if turn['p1_pokemon_state']['name'] == pok['name']:
                    last_available_state = turn['p1_pokemon_state']
            elif turn.get('turn') is not None and turn['turn'] > 30:
                # If we passed turn 30, the last available state before or at 30 is the one to use
                break

        if last_available_state:
            life_pct = last_available_state.get('hp_pct', 0.0)
            life_abs = life_pct * life
            lifes1_at_turn_30.append(life_abs)
        else:
            # If a pokemon from team1 never appeared by turn 30, assume full HP
            lifes1_at_turn_30.append(life)

    # Find the state of each pokemon in team2 at turn 30 or the last available turn before 30
    for pok in team2:
        pok2_data = pokemon_by_name[pok['name']]
        life = pok2_data['base_hp']
        last_available_state = None

        for turn in battle['battle_timeline']:
            if turn.get('turn') is not None and turn['turn'] <= 30:
                 if turn['p2_pokemon_state']['name'] == pok['name']:
                    last_available_state = turn['p2_pokemon_state']
            elif turn.get('turn') is not None and turn['turn'] > 30:
                # If we passed turn 30, the last available state before or at 30 is the one to use
                break


        if last_available_state:
            life_pct = last_available_state.get('hp_pct', 0.0)
            life_abs = life_pct * life
            lifes2_at_turn_30.append(life_abs)
        else:
            # If a pokemon from team2 never appeared by turn 30, assume full HP
            lifes2_at_turn_30.append(life)

    p1_life_turn30 = sum(lifes1_at_turn_30)
    p2_life_turn30 = sum(lifes2_at_turn_30)
    life_diff_turn30 = p1_life_turn30 - p2_life_turn30

    return p1_life_turn30, p2_life_turn30, life_diff_turn30

# Apply the function to train_data and add the feature to df
df['life_diff_turn30'] = [calculate_life_diff_turn30(battle)[2] for battle in train_data]
df['p1_life_turn30'] = [calculate_life_diff_turn30(battle)[0] for battle in train_data]
df['p2_life_turn30'] = [calculate_life_diff_turn30(battle)[1] for battle in train_data]

# Apply the function to test_data and add the feature to df_test
df_test['life_diff_turn30'] = [calculate_life_diff_turn30(battle)[2] for battle in test_data]
df_test['p1_life_turn30'] = [calculate_life_diff_turn30(battle)[0] for battle in test_data]
df_test['p2_life_turn30'] = [calculate_life_diff_turn30(battle)[1] for battle in test_data]

In [49]:
for battle in train_data:
  for pok in pokemon:
    battle[pok['name']] = 0
    if pok in battle['p1_team_details']:
      battle[pok['name']] += 1
    if pok in battle['p2_team_details']:
      battle[pok['name']] -= 1

for battle in test_data:
  for pok in pokemon:
    battle[pok['name']] = 0
    if pok in battle['p1_team_details']:
      battle[pok['name']] += 1
    if pok in battle['p2_team_details']:
      battle[pok['name']] -= 1

In [50]:
for pok in pokemon:
  df[pok['name']] = [battle[pok['name']] for battle in train_data]
  df_test[pok['name']] = [battle[pok['name']] for battle in test_data]

In [52]:
def analyze_status_duration(battle):
    p1_status_turns = 0
    p2_status_turns = 0
    p1_pokemon_status = {} # To track the status of each pokemon across turns
    p2_pokemon_status = {}

    timeline = battle.get('battle_timeline', [])

    for turn in timeline:
        p1_state = turn.get('p1_pokemon_state', {})
        p2_state = turn.get('p2_pokemon_state', {})

        # Update status for active pokemon
        p1_name = p1_state.get('name')
        p2_name = p2_state.get('name')

        if p1_name:
            p1_pokemon_status[p1_name] = p1_state.get('status', 'nostatus')
        if p2_name:
            p2_pokemon_status[p2_name] = p2_state.get('status', 'nostatus')

        # Count turns with status for any pokemon in the team that has appeared
        for name, status in p1_pokemon_status.items():
            if status != 'nostatus':
                p1_status_turns += 1

        for name, status in p2_pokemon_status.items():
            if status != 'nostatus':
                p2_status_turns += 1

    # We can also calculate the difference
    status_turns_diff = p2_status_turns - p1_status_turns

    return p1_status_turns, p2_status_turns, status_turns_diff

# Apply the function to train_data and test_data and add the features to df
df_list = df.to_dict('records')
status_features_train = [analyze_status_duration(battle) for battle in df_list]
df['p1_status_turns'] = [f[0] for f in status_features_train]
df['p2_status_turns'] = [f[1] for f in status_features_train]
df['diff_status_turns'] = [f[2] for f in status_features_train]

df_test_list = df_test.to_dict('records')
status_features_test = [analyze_status_duration(battle) for battle in df_test_list]
df_test['p1_status_turns'] = [f[0] for f in status_features_test]
df_test['p2_status_turns'] = [f[1] for f in status_features_test]
df_test['diff_status_turns'] = [f[2] for f in status_features_test]

In [73]:
#Creating interaction features and base stats feeature

#Interaction features
df['fainted_life_interaction'] = df['diff_fainted_pokemon'] * df['life_diff_turn30']
df['damage_status_interaction'] = df['damage_diff'] * df['diff_status']

df_test['fainted_life_interaction'] = df_test['diff_fainted_pokemon'] * df_test['life_diff_turn30']
df_test['damage_status_interaction'] = df_test['damage_diff'] * df_test['diff_status']

# Extract individual stat differences into separate columns
stats = ["base_hp", "base_atk", "base_def", "base_spa", "base_spd", "base_spe"]
# Calculate stat differences directly
p1_team_stats_train = df['p1_team_details'].apply(lambda team: {s: sum(p[s] for p in team) for s in stats})
p2_team_stats_train = df['p2_team_details'].apply(lambda team: {s: sum(p[s] for p in team) for s in stats})
for stat in stats:
    df[f"{stat}_p1"] = p1_team_stats_train.apply(lambda x: x[stat])
    df[f"{stat}_p2"] = p2_team_stats_train.apply(lambda x: x[stat])
    df[f"{stat}_diff"] = p1_team_stats_train.apply(lambda x: x[stat]) - p2_team_stats_train.apply(lambda x: x[stat])

p1_team_stats_test = df_test['p1_team_details'].apply(lambda team: {s: sum(p[s] for p in team) for s in stats})
p2_team_stats_test = df_test['p2_team_details'].apply(lambda team: {s: sum(p[s] for p in team) for s in stats})
for stat in stats:
    df_test[f"{stat}_p1"] = p1_team_stats_test.apply(lambda x: x[stat])
    df_test[f"{stat}_p2"] = p2_team_stats_test.apply(lambda x: x[stat])
    df_test[f"{stat}_diff"] = p1_team_stats_test.apply(lambda x: x[stat]) - p2_team_stats_test.apply(lambda x: x[stat])

# Add the new features to the list of features
features = [
    'player_won',
    'priority1', 'priority2', 'priority_diff',
    'special1', 'special2', 'special_diff',
    'mult1', 'mult2', 'mult_diff',
    'damage1', 'damage2', 'damage_diff',
    'p1_hb_ko', 'p2_hb_ko',
    'first_cc_winner', 'early_sleep_hits', 'early_freeze_hits',
    'p1_total_stab_bp', 'p2_total_stab_bp', 'diff_total_stab_bp',
    'tot_boostp1', 'tot_boostp2', 'diff_boosts',
    'p1_avg_move_accuracy', 'p2_avg_move_accuracy', 'diff_move_accuracy',
    'tot_statusp1', 'tot_statusp2', 'diff_status',
    'p1_fainted_pokemon', 'p2_fainted_pokemon', 'diff_fainted_pokemon',
    'p1_voluntary_switches', 'p2_voluntary_switches', 'diff_switches',
    'p1_healing_moves', 'p2_healing_moves', 'diff_healing_moves',
    'life_diff_turn30', 'p1_life_turn30', 'p2_life_turn30',
    'p1_status_turns', 'p2_status_turns', 'diff_status_turns',
    'base_hp_p1', 'base_hp_p2', 'base_hp_diff',
    'base_atk_p1', 'base_atk_p2', 'base_atk_diff',
    'base_def_p1', 'base_def_p2', 'base_def_diff',
    'base_spa_p1', 'base_spa_p2', 'base_spa_diff',
    'base_spd_p1', 'base_spd_p2', 'base_spd_diff',
    'base_spe_p1', 'base_spe_p2', 'base_spe_diff',
    'fainted_life_interaction',
    'damage_status_interaction',
    'type_adv',
    'starmie', 'exeggutor', 'chansey', 'snorlax', 'tauros', 'alakazam', 'jynx',
    'slowbro', 'gengar', 'rhydon', 'zapdos', 'cloyster', 'golem', 'jolteon',
    'articuno', 'persian', 'lapras', 'dragonite', 'charizard', 'victreebel']

df_selected = df[features]

features_test = features.copy()
features_test.remove('player_won')
df_test_selected = df_test[features_test]

In [85]:
correlation_matrix = df_selected.corr()

# Display the correlation of features with 'player_won'
print("Correlation with 'player_won':")
print(correlation_matrix['player_won'].sort_values(ascending=False))

Correlation with 'player_won':
player_won              1.000000
diff_status_turns       0.546225
diff_status             0.467552
p1_life_turn30          0.457538
life_diff_turn30        0.442507
                          ...   
mult2                  -0.301540
tot_statusp1           -0.364386
p1_fainted_pokemon     -0.463773
p1_status_turns        -0.468036
diff_fainted_pokemon   -0.486315
Name: player_won, Length: 86, dtype: float64


In [81]:
# Define target variable (y) and independent variables (X) for training
y = df_selected['player_won']
X_train = df_selected.drop('player_won', axis=1)

# Independent variables for testing
X_test = df_test_selected.copy()

# Scale the features for both training and test sets
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply PCA, selecting the first 35 components
pca = PCA(n_components=35)

# Fit PCA on the scaled training data and transform both training and test data
X_train_pca = pca.fit_transform(X_scaled)
X_test_pca = pca.transform(X_test_scaled)

print(f"Shape of X_train_pca: {X_train_pca.shape}")
print(f"Shape of X_test_pca: {X_test_pca.shape}")

Shape of X_train_pca: (9986, 35)
Shape of X_test_pca: (5000, 35)


# Logistic Model


In [93]:
# Instantiate a LogisticRegression object
model = LogisticRegression()

In [94]:
model.fit(X_train_pca, y)

LogisticRegression()

In [95]:
# Make predictions
y_pred = model.predict(X_train_pca)

# Calculate evaluation metrics
accuracy = accuracy_score(y, y_pred)
precision = precision_score(y, y_pred)
recall = recall_score(y, y_pred)
f1 = f1_score(y, y_pred)

# Print the metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Accuracy: 0.8327
Precision: 0.8281
Recall: 0.8396
F1-score: 0.8338


# K-Fold Validation

In [100]:
# Define the number of folds
n_splits = 10

# Instantiate StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# List to store accuracy scores for each fold
accuracy_scores = []

# Perform K-fold cross-validation
for train_index, val_index in skf.split(X_train_pca, y):
    X_train_fold, X_val_fold = X_train_pca[train_index], X_train_pca[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]

    # Instantiate and train the model
    model = LogisticRegression(penalty='l2', solver='liblinear', random_state=42)
    model.fit(X_train_fold, y_train_fold)

    # Predict on the validation set
    y_pred_fold = model.predict(X_val_fold)

    # Calculate accuracy for the current fold and append to the list
    fold_accuracy = accuracy_score(y_val_fold, y_pred_fold)
    accuracy_scores.append(fold_accuracy)

# Calculate the average accuracy across all folds
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)

print(f"Accuracy scores for each fold: {accuracy_scores}")
print(f"Mean Accuracy: {mean_accuracy:.4f}")
print(f"Standard Deviation of Accuracy: {std_accuracy:.4f}")

Accuracy scores for each fold: [0.8348348348348348, 0.8298298298298298, 0.8268268268268268, 0.8248248248248248, 0.8428428428428428, 0.8278278278278278, 0.8346693386773547, 0.8066132264529058, 0.8366733466933868, 0.842685370741483]
Mean Accuracy: 0.8308
Standard Deviation of Accuracy: 0.0100


# Stacking Model

In [101]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

# Initialize base models with default parameters
log_reg_model = LogisticRegression(random_state=42)
rf_model = RandomForestClassifier(random_state=42)
gbm_model = GradientBoostingClassifier(random_state=42)
xgb_model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
knn_model = KNeighborsClassifier(n_neighbors=5)

In [102]:
from sklearn.ensemble import StackingClassifier

# Define base models for stacking
estimators = [
    ('lr', log_reg_model),
    ('rf', rf_model),
    ('gbm', gbm_model),
    ('xgb', xgb_model),
    ('knn', knn_model) # Add KNN model
]

# Define the final estimator (meta-learner)
final_estimator = LogisticRegression(random_state=42)

# Instantiate the StackingClassifier
stacking_classifier = StackingClassifier(
    estimators=estimators,
    final_estimator=final_estimator,
    cv=5, # Using 5-fold cross-validation for meta-learner training
    n_jobs=-1, # Use all available cores
    passthrough=True # Pass original features to the final estimator
)

In [103]:
# Define base models for stacking
estimators = [
    ('lr', log_reg_model),
    ('rf', rf_model),
    ('gbm', gbm_model),
    ('xgb', xgb_model),
    ('knn', knn_model)
]

# Define the model of final estimator (meta-learner)
final_estimator = LogisticRegression(random_state=42)


stacking_classifier = StackingClassifier(
    estimators=estimators,
    final_estimator=final_estimator,
    cv=5, # Using 5-fold cross-validation for meta-learner training
    n_jobs=-1, # Use all available cores
    passthrough=True # Pass original features to the final estimator
)

print("StackingClassifier configured successfully.")

StackingClassifier configured successfully.


In [105]:
print("Training StackingClassifier...")
stacking_classifier.fit(X_train_pca, y)
print("StackingClassifier training complete.")

Training StackingClassifier...
StackingClassifier training complete.


In [106]:
# Make predictions on the training data using the trained StackingClassifier
y_pred_stacking_train = stacking_classifier.predict(X_train_pca)

# Calculate the accuracy score
accuracy_stacking = accuracy_score(y, y_pred_stacking_train)

print(f"Accuracy of the StackingClassifier on the training set: {accuracy_stacking:.4f}")

Accuracy of the StackingClassifier on the training set: 0.8814


# Making Predictions

In [108]:
# Make predictions on the scaled test data
predictions = stacking_classifier.predict(X_test_pca)

# Create a submission DataFrame
submission_df = pd.DataFrame({'battle_id': df_test['battle_id'], 'player_won': predictions})

# Convert boolean predictions to 0/1 integers
submission_df['player_won'] = submission_df['player_won'].astype(int)

# Display the first few rows of the submission DataFrame
display(submission_df.head())

# Save the submission file to CSV
submission_df.to_csv('submission.csv', index=False)
print("Submission file 'submission.csv' created successfully!")

,battle_id,player_won
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1


Submission file 'submission.csv' created successfully!


# Kfold stacking model


In [109]:
# Define the number of folds
n_splits = 10

# Instantiate StratifiedKFold with shuffle and random_state for reproducibility
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

print(f"Stratified K-Fold cross-validation object initialized with {n_splits} splits.")

Stratified K-Fold cross-validation object initialized with 10 splits.


In [110]:
# List to store accuracy scores for each fold
accuracy_scores_stacking = []

# Perform K-fold cross-validation
print(f"Performing {n_splits}-fold cross-validation for StackingClassifier...")
for fold, (train_index, test_index) in enumerate(skf.split(X_scaled, y)):
    print(f"\n--- Fold {fold+1}/{n_splits} ---")
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Instantiate a fresh StackingClassifier for each fold to ensure independent training
    # This is important to prevent information leakage between folds through the meta-learner.
    fold_stacking_classifier = StackingClassifier(
        estimators=estimators,
        final_estimator=final_estimator,
        cv=5, # Using 5-fold cross-validation for meta-learner training
        n_jobs=-1, # Use all available cores
        passthrough=True
    )

    # Train the StackingClassifier on the current fold's training data
    print(f"Training StackingClassifier for Fold {fold+1}...")
    fold_stacking_classifier.fit(X_train, y_train)

    # Predict on the test set for the current fold
    y_pred_fold = fold_stacking_classifier.predict(X_test)

    # Calculate accuracy for the current fold and append to the list
    fold_accuracy = accuracy_score(y_test, y_pred_fold)
    accuracy_scores_stacking.append(fold_accuracy)
    print(f"Accuracy for Fold {fold+1}: {fold_accuracy:.4f}")

# Calculate the average accuracy across all folds
mean_accuracy_stacking = np.mean(accuracy_scores_stacking)
std_accuracy_stacking = np.std(accuracy_scores_stacking)

print(f"\nAccuracy scores for each fold: {accuracy_scores_stacking}")
print(f"Mean StackingClassifier Accuracy: {mean_accuracy_stacking:.4f}")
print(f"Standard Deviation of StackingClassifier Accuracy: {std_accuracy_stacking:.4f}")

Performing 10-fold cross-validation for StackingClassifier...

--- Fold 1/10 ---
Training StackingClassifier for Fold 1...
Accuracy for Fold 1: 0.8238

--- Fold 2/10 ---
Training StackingClassifier for Fold 2...
Accuracy for Fold 2: 0.8318

--- Fold 3/10 ---
Training StackingClassifier for Fold 3...
Accuracy for Fold 3: 0.8318

--- Fold 4/10 ---
Training StackingClassifier for Fold 4...
Accuracy for Fold 4: 0.8268

--- Fold 5/10 ---
Training StackingClassifier for Fold 5...
Accuracy for Fold 5: 0.8589

--- Fold 6/10 ---
Training StackingClassifier for Fold 6...
Accuracy for Fold 6: 0.8288

--- Fold 7/10 ---
Training StackingClassifier for Fold 7...
Accuracy for Fold 7: 0.8297

--- Fold 8/10 ---
Training StackingClassifier for Fold 8...
Accuracy for Fold 8: 0.8136

--- Fold 9/10 ---
Training StackingClassifier for Fold 9...
Accuracy for Fold 9: 0.8377

--- Fold 10/10 ---
Training StackingClassifier for Fold 10...
Accuracy for Fold 10: 0.8507

Accuracy scores for each fold: [0.8238238238